<a href="https://colab.research.google.com/github/rakshithcgowda/Machine-project/blob/main/Mean_variance_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import scipy.optimize as sco
import matplotlib.pyplot as plt

# Example: Sample portfolio data with random stock returns
np.random.seed(42)
returns = np.random.normal(0.001, 0.01, (1000, 4))  # Simulate daily returns for 4 assets over 1000 days
assets = ['Stock A', 'Stock B', 'Stock C', 'Stock D']
df = pd.DataFrame(returns, columns=assets)

# Calculate mean returns and covariance matrix
mean_returns = df.mean() * 252  # Annualized mean returns (assuming 252 trading days)
cov_matrix = df.cov() * 252  # Annualized covariance matrix

# Define functions for portfolio performance: returns and volatility (standard deviation)
def portfolio_performance(weights, mean_returns, cov_matrix):
    portfolio_return = np.sum(weights * mean_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return portfolio_return, portfolio_volatility

# Define objective function: minimize volatility (risk)
def minimize_volatility(weights, mean_returns, cov_matrix):
    return portfolio_performance(weights, mean_returns, cov_matrix)[1]

# Constraints: Sum of weights must equal 1 (fully invested)
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for _ in range(len(assets)))  # Weights between 0 and 1

# Initial guess for weights
initial_guess = len(assets) * [1. / len(assets)]  # Equal weight distribution

# Optimization: Minimize risk (volatility) subject to the constraints
result = sco.minimize(minimize_volatility, initial_guess, args=(mean_returns, cov_matrix),
                      method='SLSQP', bounds=bounds, constraints=constraints)

# Extract the optimal weights
optimal_weights = result.x

# Calculate optimal portfolio performance
optimal_return, optimal_volatility = portfolio_performance(optimal_weights, mean_returns, cov_matrix)

# Display results
print("Optimal Weights:", dict(zip(assets, optimal_weights)))
print("Expected Portfolio Return: {:.2f}%".format(optimal_return * 100))
print("Expected Portfolio Volatility (Risk): {:.2f}%".format(optimal_volatility * 100))

# Visualize optimal portfolio weights
plt.figure(figsize=(10, 6))
plt.pie(optimal_weights, labels=assets, autopct='%1.1f%%', startangle=90)
plt.title("Optimal Portfolio Allocation")
plt.show()